In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date, lit

In [0]:
df_faturas = (
    spark.read.table("production.raw.tb_faturas")
    .select(
        F.col("fatura_id"),
        F.col("reserva_id"),
        F.col("data_cobranca"),
        F.round(F.col("valor_total").cast("double"), 2).alias("valor_total"),
        F.col("impostos"),
        F.col("forma_pagamento"),
        F.col("status_pagamento")
    )
    .withColumn("data_cobranca", F.to_date(F.col("data_cobranca")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_faturas.limit(5))


In [0]:
catalog_table = "production.trusted.tb_faturas"

if spark.catalog.tableExists(catalog_table):
    delta_table = DeltaTable.forName(spark, catalog_table)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_faturas.alias("source"),
        "target.consumo_id = source.consumo_id"
    ).whenMatchedUpdate(
        set = {
            **{col: f"source.{col}" for col in df_faturas.columns if col != "update_date"},
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    (
        df_faturas
        .withColumn("update_date", lit(""))
        .write
        .format("delta")
        .saveAsTable(catalog_table)
    )